In [1]:
import datetime as dt
import polars as pl
import factor_momentum as fm

from sf_quant.data import load_assets

from factor_momentum._loaders import _load_monthly_asset_data

/home/boobus/projects/factor_momentum/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-27 08:47:48,596	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
start = dt.date(2001,1,1)
end = dt.date(2020,1,1)

TYPE = 'rolling continuous'

In [3]:
signal = fm.assetspace_signal_monthly(start=start, end=end, type=TYPE)
signal

DEBUG: New


month,barrid,ret,specific_risk,market_cap,signal
date,str,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.001596,55.056916,6.006157e8,-0.188481
2013-08-01,"""USA06Z1""",-0.014481,54.406366,5.9463375e8,-0.059871
2013-09-01,"""USA06Z1""",-0.391781,57.884562,4.0188375e8,-0.062705
2013-10-01,"""USA06Z1""",0.23979,58.065759,5.174708e8,-0.017102
2013-11-01,"""USA06Z1""",0.172845,57.269027,6.071625e8,0.103536
…,…,…,…,…,…
2019-12-01,"""USBEVY1""",0.038019,77.098934,6.0276e8,-0.190582
2019-12-01,"""USBEWD1""",0.007976,67.137569,2.0000e8,-0.18278
2019-12-01,"""USBEWE1""",-0.030714,62.962481,3.6251e8,-0.125273


In [11]:
(signal.with_columns(
    (pl.col('signal').sub(pl.col('signal').mean().over('month')) / pl.col('signal').std().over('month')).alias('score'),
    pl.col('signal').mul(0).add(0.05).alias('IC'),
    pl.col('specific_risk').mul(0.01)
)
.with_columns(
    pl.col('score').mul(pl.col('IC')).mul(pl.col('specific_risk')).alias('alpha')
)
)

month,barrid,ret,specific_risk,market_cap,signal,score,IC,alpha
date,str,f64,f64,f64,f64,f64,f64,f64
2013-07-01,"""USA06Z1""",-0.001596,0.550569,6.006157e8,-0.188481,-2.220399,0.05,-0.061124
2013-08-01,"""USA06Z1""",-0.014481,0.544064,5.9463375e8,-0.059871,-1.404449,0.05,-0.038205
2013-09-01,"""USA06Z1""",-0.391781,0.578846,4.0188375e8,-0.062705,-1.738631,0.05,-0.05032
2013-10-01,"""USA06Z1""",0.23979,0.580658,5.174708e8,-0.017102,-0.942534,0.05,-0.027364
2013-11-01,"""USA06Z1""",0.172845,0.57269,6.071625e8,0.103536,0.700179,0.05,0.020049
…,…,…,…,…,…,…,…,…
2019-12-01,"""USBEVY1""",0.038019,0.770989,6.0276e8,-0.190582,-1.037994,0.05,-0.040014
2019-12-01,"""USBEWD1""",0.007976,0.671376,2.0000e8,-0.18278,-0.957134,0.05,-0.03213
2019-12-01,"""USBEWE1""",-0.030714,0.629625,3.6251e8,-0.125273,-0.361173,0.05,-0.01137


In [4]:
signal.with_columns(
    (1).alias('1')
)

AttributeError: 'int' object has no attribute 'alias'